## Prototipo di gestione password su ambienti Microsoft SQL Server 

<span style="font-size: 14px;">Autori: Simone Greci, Leonardo Contini, Donato Chiarello</span>

Creazione database:

In [ ]:
create database AuthorizationConfig
go

Creazione tabella di supporto:

In [ ]:
use AuthorizationConfig
go

create table passwords_config
(
    id int identity(1,1) primary key,
    server_name varchar(255) not null, --FQDN/IP Address of the database server
    server_type varchar(10) not NULL, --describe the type os database server (eg: Orale, SQLServer, etc...)
    username varchar(255) not null, --username
    keepass_entry varchar(255) not null, --keepass entry name for the user
    ssis_parameter_name varchar(255) null, --ssis parameter name mapped for this user
    keepass_file_db_path_1 varchar(1024) null, --path of the 1st database file
    keepass_file_key_path_1 varchar(1024) null, --path of the 1st key file
    keepass_file_db_path_2 varchar(1024) null, --path of the 2nd database file
    keepass_file_key_path_2 varchar(1024) null --path of the 2nd key file
)

#create unique constraint
CREATE UNIQUE NONCLUSTERED INDEX IX_passwords_config_servername_username ON passwords_config (server_name, username)

#insert test data
INSERT INTO passwords_config ([server_name],[server_type],[username],[keepass_entry],[ssis_parameter_name],[keepass_file_db_path_1],[keepass_file_key_path_1],[keepass_file_db_path_2],[keepass_file_key_path_2])
VALUES ('localhost','SQLServer','localhost_user1','user1','localhost_user1','C:\Projects\database1.keyx','C:\Projects\database1.keyx','C:\Projects\database2.keyx','C:\Projects\database2.keyx')


Esempi di utilizzo commandlets Keepass disponibili su GitHub:

https://github.com/PSKeePass/PoShKeePass

In [ ]:
#Install PoShKeePass commandlets
if($null -eq (Get-InstalledModule PoShKeePass))
{
    Install-Module PoShKeePass
}

#Create a new configuration
if($null -eq (Get-KeePassDatabaseConfiguration -DatabaseProfileName ssis))
{
    New-KeePassDatabaseConfiguration -DatabasePath "C:\Projects\database1.kdbx" -DatabaseProfileName "ssis" -KeyPath "C:\Projects\database1.keyx"
}

#Get the configuration
Get-KeePassDatabaseConfiguration -DatabaseProfileName ssis

#Read entry from database
Get-KeePassEntry -DatabaseProfileName ssis -Title "localhost_user1" -AsPlainText

Utilizzo modulo "SimplySQL" -\> https://github.com/mithrandyr/SimplySql per interrogare il database:

In [1]:
if($null -eq (Get-InstalledModule SimplySql))
{
    Install-Module SimplySql
}

#open connectino to database:
Open-SqlConnection -ConnectionName "sqlserver" -Server localhost -Database "AuthorizationConfig"

#open record
$results = Invoke-SqlQuery -ConnectionName "sqlserver" -Query "SELECT TOP 1 * FROM dbo.passwords_config"

foreach($row in $results)
{
   Write-Host $row["keepass_entry"]
}

user1
